In [24]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as dc
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [25]:
# Load the dataset
data = pd.read_csv('Biotech_preprocessed_dataset.csv')
data

,Time,TOD,Total (W),Phase_A (W),Phase_B (W),Phase_C (W),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday
0,2024-01-04 15:00:00,15:00:00,821.0,207.0,467.0,147.0,False,False,False,False,True,False,False,False
1,2024-01-04 16:00:00,16:00:00,741.0,298.0,329.0,114.0,False,False,False,False,True,False,False,False
2,2024-01-04 17:00:00,17:00:00,776.0,363.0,278.0,135.0,False,False,False,False,True,False,False,False
3,2024-01-04 18:00:00,18:00:00,1260.0,659.0,419.0,182.0,False,False,False,False,True,False,False,False
4,2024-01-04 19:00:00,19:00:00,973.0,381.0,410.0,182.0,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4454,2024-07-08 05:00:00,05:00:00,1040.0,635.0,259.0,146.0,False,True,False,False,False,False,False,False
4455,2024-07-08 06:00:00,06:00:00,1414.0,1023.0,251.0,140.0,False,True,False,False,False,False,False,False
4456,2024-07-08 07:00:00,07:00:00,571.0,225.0,243.0,103.0,False,True,False,False,False,False,False,False
4457,2024-07-08 08:00:00,08:00:00,784.0,411.0,280.0,93.0,False,True,False,False,False,False,False,False


In [26]:
# To keep it simple, we will predict only on the total active power first
data.drop(['Phase_A (W)', 'Phase_B (W)', 'Phase_C (W)'], axis=1, inplace=True)
data

,Time,TOD,Total (W),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday
0,2024-01-04 15:00:00,15:00:00,821.0,False,False,False,False,True,False,False,False
1,2024-01-04 16:00:00,16:00:00,741.0,False,False,False,False,True,False,False,False
2,2024-01-04 17:00:00,17:00:00,776.0,False,False,False,False,True,False,False,False
3,2024-01-04 18:00:00,18:00:00,1260.0,False,False,False,False,True,False,False,False
4,2024-01-04 19:00:00,19:00:00,973.0,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
4454,2024-07-08 05:00:00,05:00:00,1040.0,False,True,False,False,False,False,False,False
4455,2024-07-08 06:00:00,06:00:00,1414.0,False,True,False,False,False,False,False,False
4456,2024-07-08 07:00:00,07:00:00,571.0,False,True,False,False,False,False,False,False
4457,2024-07-08 08:00:00,08:00:00,784.0,False,True,False,False,False,False,False,False


In [27]:
# Bring the Total (W) column to the last
power_col = data.pop('Total (W)')
data.insert(len(data.columns), power_col.name, power_col)
data

,Time,TOD,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday,Total (W)
0,2024-01-04 15:00:00,15:00:00,False,False,False,False,True,False,False,False,821.0
1,2024-01-04 16:00:00,16:00:00,False,False,False,False,True,False,False,False,741.0
2,2024-01-04 17:00:00,17:00:00,False,False,False,False,True,False,False,False,776.0
3,2024-01-04 18:00:00,18:00:00,False,False,False,False,True,False,False,False,1260.0
4,2024-01-04 19:00:00,19:00:00,False,False,False,False,True,False,False,False,973.0
...,...,...,...,...,...,...,...,...,...,...,...
4454,2024-07-08 05:00:00,05:00:00,False,True,False,False,False,False,False,False,1040.0
4455,2024-07-08 06:00:00,06:00:00,False,True,False,False,False,False,False,False,1414.0
4456,2024-07-08 07:00:00,07:00:00,False,True,False,False,False,False,False,False,571.0
4457,2024-07-08 08:00:00,08:00:00,False,True,False,False,False,False,False,False,784.0


In [28]:
# extract hour from 'TOD'
data['Time'] = pd.to_datetime(data['Time'])
data['TOD'] = data['Time'].dt.hour

# Since we need only the hour of day, drop the 'Time column'
data.drop('Time', axis=1, inplace=True)
data

,TOD,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday,Total (W)
0,15,False,False,False,False,True,False,False,False,821.0
1,16,False,False,False,False,True,False,False,False,741.0
2,17,False,False,False,False,True,False,False,False,776.0
3,18,False,False,False,False,True,False,False,False,1260.0
4,19,False,False,False,False,True,False,False,False,973.0
...,...,...,...,...,...,...,...,...,...,...
4454,5,False,True,False,False,False,False,False,False,1040.0
4455,6,False,True,False,False,False,False,False,False,1414.0
4456,7,False,True,False,False,False,False,False,False,571.0
4457,8,False,True,False,False,False,False,False,False,784.0


In [29]:
# set up the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [61]:
# convert to sequences
def create_sequences(data, labels, seq_length):
  sequences = []
  targets = []
  for i in range(len(data)-seq_length):
    seq = data[i:i+seq_length]
    label = labels[i+seq_length]
    sequences.append(seq)
    targets.append(label)
  return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

In [59]:
features = data.iloc[:, :-1].values
labels = data.iloc[:, -1].values

scaler = MinMaxScaler(feature_range=(0, 1))
features = scaler.fit_transform(features)

In [72]:
SEQ_LENGTH = 10
sequences, targets = create_sequences(features, labels, SEQ_LENGTH)
batch_size = len(sequences)
print(sequences.shape, targets.shape)
print(batch_size)

torch.Size([4449, 10, 9]) torch.Size([4449])
4449


In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [74]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3567, 9), (892, 9), (3567,), (892,))

In [75]:
# torch dataset
class TimeSeriesDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, i):
    return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [79]:
# Dataloader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [98]:
# LSTM model
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_stacked_layers):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_stacked_layers = num_stacked_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x):
    batch_size = x.size(0)
    print(x.shape[0])
    print("Data type of x: ", x.dtype)
    # h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
    # c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
    h0 = torch.zeros(self.num_stacked_layers, self.hidden_size).to(device)
    c0 = torch.zeros(self.num_stacked_layers, self.hidden_size).to(device)

    x = x.float()
    print("Data type of x after: ", x.dtype)
    out, _ = self.lstm(x.float(), (h0, c0))
    out = self.fc(out[:, -1])
    return out

input_size = features.shape[1]
hidden_size = 32
# output_size = 1
num_stacked_layers = 1
model = LSTM(input_size, hidden_size, num_stacked_layers)
model.to(device)
model

LSTM(
  (lstm): LSTM(9, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

# Train the Model

In [99]:
crieterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
model.train()

for epoch in range(num_epochs):
  for sequences, targets in train_loader:
    sequences = sequences.to(device)
    targets = targets.to(device)

    outputs = model(sequences)
    loss = crieterion(outputs, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

32
Data type of x:  torch.float64
Data type of x after:  torch.float32


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Double but expected Float

In [70]:
model.eval()
with torch.no_grad():
    test_loss = 0
    for sequences, targets in test_loader:
        sequences, targets = sequences.to(device), targets.to(device)
        outputs = model(sequences)
        loss = crieterion(outputs, targets)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss / len(test_loader):.4f}')

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [38]:
for _, batch in enumerate(train_loader):
  x_batch, y_batch = batch[0].to(device), batch[1].to(device)
  print(x_batch.shape, y_batch.shape)
  break


torch.Size([32, 9]) torch.Size([32])


In [56]:
def train_one_epoch():
  model.train(True)
  print(f"Epoch: {epoch+1}")
  running_loss = 0.0
  for batch_index, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)
    loss = loss_function(output, y_batch)
    running_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_index % 100 == 99:
      avg_loss_across_batches = running_loss/100
      print(f"Batch {batch_index+1} loss: {avg_loss_across_batches}")
      running_loss = 0.0

    print()


In [57]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [58]:
learning_rate = 0.001
num_epochs = 10
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1


RuntimeError: Expected hidden[0] size (1, 1, 4), got [32, 1, 4]